# NLP Preprocessing

In [1]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai import *

`text.tranform` contains the functions that deal behind the scenes with the two main tasks when preparing texts for modelling: *tokenization* and *numericalization*.

*Tokenization* splits the raw texts into tokens (wich can be words, or punctuation signs...). The most basic way to do this would be to separate according to spaces, but it's possible to be more subtle; for instance, the contractions like "isn't" or "don't" should be split in \["is","n't"\] or \["do","n't"\]. By default fastai will use the powerful [spacy tokenizer](https://spacy.io/api/tokenizer).

*Numericalization* is easier as it just consists in attributing a unique id to each token and mapping each of those tokens to their respective ids.

## Tokenization

### Introduction

This step is actually divided in two phases: first, we apply a certain list of `rules` to the raw texts as preprocessing, then we use the tokenizer to split them in lists of tokens. Combining together those `rules`, the `tok_func`and the `lang` to process the texts is the role of the [`Tokenizer`](/text.transform.html#Tokenizer) class.

In [2]:
show_doc(Tokenizer, doc_string=False)

<h2 id="Tokenizer"><code>class</code> <code>Tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L74" class="source_link">[source]</a></h2>

> <code>Tokenizer</code>(`tok_func`:`Callable`=`'SpacyTokenizer'`, `lang`:`str`=`'en'`, `rules`:`ListRules`=`None`, `special_cases`:`StrList`=`None`, `n_cpus`:`int`=`None`)

This class will process texts by appling them the `rules` then tokenizing them with `tok_func(lang)`. `special_cases` are a list of tokens passed as special to the tokenizer and `n_cpus` is the number of cpus to use for multi-processing (by default, half the cpus available). We don't directly pass a tokenizer for multi-processing purposes: each process needs to initiate a tokenizer of its own. The rules and special_cases default to

`default_rules = [fix_html, replace_rep, replace_wrep, deal_caps, spec_add_spaces, rm_useless_spaces]` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L78">[source]</a></div>

and

`default_spec_tok = [BOS, FLD, UNK, PAD]` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L79">[source]</a></div>

In [3]:
show_doc(Tokenizer.process_text)

<h4 id="Tokenizer.process_text"><code>process_text</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L88" class="source_link">[source]</a></h4>

> <code>process_text</code>(`t`:`str`, `tok`:[`BaseTokenizer`](/text.transform.html#BaseTokenizer)) → `List`\[`str`\]

Processe one text `t` with tokenizer `tok`.  

In [4]:
show_doc(Tokenizer.process_all)

<h4 id="Tokenizer.process_all"><code>process_all</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L99" class="source_link">[source]</a></h4>

> <code>process_all</code>(`texts`:`StrList`) → `List`\[`List`\[`str`\]\]

Process a list of `texts`.  

For an example, we're going to grab some IMDB reviews.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/fastai/fastai/../data/imdb_sample')

In [ ]:
df = pd.read_csv(path/'train.csv', header=None)
example_text = df.iloc[2][1]; example_text

'Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt like I was watching a junior high video presentation. Have the directors, producers, etc. ever even seen a movie before? Halestorm is getting worse and worse with every new entry. The concept for this movie sounded so funny. How could you go wrong with Gary Coleman and a handful of somewhat legitimate actors. But trust me when I say this, things went wrong, VERY WRONG.'

In [ ]:
tokenizer = Tokenizer()
tok = SpacyTokenizer('en')
' '.join(tokenizer.process_text(example_text, tok))

'every once in a long while a movie will come along that will be so awful that i feel compelled to warn people . if i labor all my days and i can save but one soul from watching this movie , how great will be my joy . \n\n where to begin my discussion of pain . for starters , there was a musical montage every five minutes . there was no character development . every character was a stereotype . we had swearing guy , fat guy who eats donuts , goofy foreign guy , etc . the script felt as if it were being written as the movie was being shot . the production value was so incredibly low that it felt like i was watching a junior high video presentation . have the directors , producers , etc . ever even seen a movie before ? halestorm is getting worse and worse with every new entry . the concept for this movie sounded so funny . how could you go wrong with gary coleman and a handful of somewhat legitimate actors . but trust me when i say this , things went wrong , xxup very xxup wrong .'

As explained before, the tokenizer split the text according to words/punctuations signs but in a smart manner. The rules (see below) also have modified the text a little bit. We can tokenize a list of texts directly at the same time:

In [ ]:
df = pd.read_csv(path/'train.csv', header=None)
texts = df[1].values
tokenizer = Tokenizer()
tokens = tokenizer.process_all(texts)
' '.join(tokens[2])

'every once in a long while a movie will come along that will be so awful that i feel compelled to warn people . if i labor all my days and i can save but one soul from watching this movie , how great will be my joy . \n\n where to begin my discussion of pain . for starters , there was a musical montage every five minutes . there was no character development . every character was a stereotype . we had swearing guy , fat guy who eats donuts , goofy foreign guy , etc . the script felt as if it were being written as the movie was being shot . the production value was so incredibly low that it felt like i was watching a junior high video presentation . have the directors , producers , etc . ever even seen a movie before ? halestorm is getting worse and worse with every new entry . the concept for this movie sounded so funny . how could you go wrong with gary coleman and a handful of somewhat legitimate actors . but trust me when i say this , things went wrong , xxup very xxup wrong .'

### Customize the tokenizer

The `tok_func` must return an instance of [`BaseTokenizer`](/text.transform.html#BaseTokenizer):

In [5]:
show_doc(BaseTokenizer)

<h2 id="BaseTokenizer"><code>class</code> <code>BaseTokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L12" class="source_link">[source]</a></h2>

> <code>BaseTokenizer</code>(`lang`:`str`)

Basic class for a tokenizer function.  

In [6]:
show_doc(BaseTokenizer.tokenizer)

<h4 id="BaseTokenizer.tokenizer"><code>tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L15" class="source_link">[source]</a></h4>

> <code>tokenizer</code>(`t`:`str`) → `List`\[`str`\]

Take a text `t` and returns the list of its tokens.

In [7]:
show_doc(BaseTokenizer.add_special_cases)

<h4 id="BaseTokenizer.add_special_cases"><code>add_special_cases</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L16" class="source_link">[source]</a></h4>

> <code>add_special_cases</code>(`toks`:`StrList`)

Record a list of special tokens `toks`.

The fastai library uses [spacy](https://spacy.io/) tokenizers as its default. The following class wraps it as [`BaseTokenizer`](/text.transform.html#BaseTokenizer).

In [8]:
show_doc(SpacyTokenizer)

<h2 id="SpacyTokenizer"><code>class</code> <code>SpacyTokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L18" class="source_link">[source]</a></h2>

> <code>SpacyTokenizer</code>(`lang`:`str`) :: [`BaseTokenizer`](/text.transform.html#BaseTokenizer)

Wrapper around a spacy tokenizer to make it a [`BaseTokenizer`](/text.transform.html#BaseTokenizer).  

If you want to use your custom tokenizer, just subclass the [`BaseTokenizer`](/text.transform.html#BaseTokenizer) and override its `tokenizer` and `add_spec_cases` functions.

### Rules

Rules are just functions that take a string and return the modified string. This allows you to customize the list of `default_rules` as you please. Those `default_rules` are:

In [9]:
show_doc(deal_caps, doc_string=False)

<h4 id="deal_caps"><code>deal_caps</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L55" class="source_link">[source]</a></h4>

> <code>deal_caps</code>(`t`:`str`) → `str`

In `t`, if a word is written in all caps, we put it in a lower case and add a special token before. A model will more easily learn this way the meaning of the sentence. The rest of the capitals are removed.

In [ ]:
deal_caps("I'm suddenly SHOUTING FOR NO REASON!")

"i'm suddenly  xxup shouting  xxup for no  xxup reason!"

In [10]:
show_doc(fix_html, doc_string=False)

<h4 id="fix_html"><code>fix_html</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L62" class="source_link">[source]</a></h4>

> <code>fix_html</code>(`x`:`str`) → `str`

This rules replaces a bunch of HTML characters or norms in plain text ones. For instance `<br />` are replaced by `\n`, `&nbsp;` by spaces etc...

In [ ]:
fix_html("Some HTML&nbsp;text<br />")

'Some HTML& text\n'

In [11]:
show_doc(replace_rep, doc_string=False)

<h4 id="replace_rep"><code>replace_rep</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L39" class="source_link">[source]</a></h4>

> <code>replace_rep</code>(`t`:`str`) → `str`

Whenever a character is repeated more than three times in `t`, we replace the whole thing by 'TK_REP n char' where n is the number of occurences and char the character.

In [ ]:
replace_rep("I'm so excited!!!!!!!!")

"I'm so excited xxrep 8 ! "

In [12]:
show_doc(replace_wrep, doc_string=False)

<h4 id="replace_wrep"><code>replace_wrep</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L47" class="source_link">[source]</a></h4>

> <code>replace_wrep</code>(`t`:`str`) → `str`

Whenever a word is repeated more than four times in `t`, we replace the whole thing by 'TK_WREP n w' where n is the number of occurences and w the word repeated.

In [ ]:
replace_wrep("I've never ever ever ever ever ever ever ever done this.")

"I've never  xxwrep 7 ever  done this."

In [13]:
show_doc(rm_useless_spaces)

<h4 id="rm_useless_spaces"><code>rm_useless_spaces</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L35" class="source_link">[source]</a></h4>

> <code>rm_useless_spaces</code>(`t`:`str`) → `str`

Remove multiple spaces in `t`.  

In [ ]:
rm_useless_spaces("Inconsistent   use  of     spaces.")

'Inconsistent use of spaces.'

In [14]:
show_doc(spec_add_spaces)

<h4 id="spec_add_spaces"><code>spec_add_spaces</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L31" class="source_link">[source]</a></h4>

> <code>spec_add_spaces</code>(`t`:`str`) → `str`

Add spaces around / and # in `t`.  

In [ ]:
spec_add_spaces('I #like to #put #hashtags #everywhere!')

'I  # like to  # put  # hashtags  # everywhere!'

## Numericalization

To convert our set of tokens to unique ids (and be able to have them go through embeddings), we use the following class:

In [15]:
show_doc(Vocab, doc_string=False)

<h2 id="Vocab"><code>class</code> <code>Vocab</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L105" class="source_link">[source]</a></h2>

> <code>Vocab</code>(`path`:`PathOrStr`)

Contain the correspondance between numbers and tokens and numericalize. `path` should point to the 'tmp' directory with the token and id files.

In [16]:
show_doc(Vocab.create, doc_string=False)

<h4 id="Vocab.create"><code>create</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L120" class="source_link">[source]</a></h4>

> <code>create</code>(`path`:`PathOrStr`, `tokens`:`Tokens`, `max_vocab`:`int`, `min_freq`:`int`) → `Vocab`

Create a [`Vocab`](/text.transform.html#Vocab) dictionary from a set of `tokens` in `path`. Only keeps `max_vocab` tokens, and only if they appear at least `min_freq` times, set the rest to `UNK`.

In [17]:
show_doc(Vocab.numericalize)

<h4 id="Vocab.numericalize"><code>numericalize</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L112" class="source_link">[source]</a></h4>

> <code>numericalize</code>(`t`:`StrList`) → `List`\[`int`\]

Convert a list of tokens `t` to their ids.  

In [18]:
show_doc(Vocab.textify)

<h4 id="Vocab.textify"><code>textify</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L116" class="source_link">[source]</a></h4>

> <code>textify</code>(`nums`:`Collection`\[`int`\]) → `List`\[`str`\]

Convert a list of `nums` to their tokens.  

In [ ]:
vocab = Vocab.create(path, tokens, max_vocab=1000, min_freq=2)
vocab.numericalize(tokens[2])[:10]

[207, 321, 11, 6, 246, 144, 6, 22, 88, 240]

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [19]:
show_doc(SpacyTokenizer.tokenizer)

<h4 id="SpacyTokenizer.tokenizer"><code>tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L24" class="source_link">[source]</a></h4>

> <code>tokenizer</code>(`t`:`str`) → `List`\[`str`\]

In [20]:
show_doc(SpacyTokenizer.add_special_cases)

<h4 id="SpacyTokenizer.add_special_cases"><code>add_special_cases</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L27" class="source_link">[source]</a></h4>

> <code>add_special_cases</code>(`toks`:`StrList`)